In [1]:
from helpers import datastore, registry
import mdsisclienttools.auth.TokenManager as ProvenaAuth
from utils import pprint_json
from env_setup import get_auth, registry_endpoint, data_store_endpoint

No storage or object provided, using default location: .tokens.json.
Using storage type: FILE.
Using DEVICE auth flow.
Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Found tokens valid, using.



In [2]:
dataset_id = "10378.1/1764777"
reason="For demonstration Purposes, creating a second version of the dataset."
version_response = datastore.version_dataset(datastore_endpoint=data_store_endpoint, dataset_id=dataset_id, reason=reason, auth=get_auth())
pprint_json(version_response)


Versioning dataset with id: 10378.1/1764777...
{
  "new_version_id": "10378.1/1764778",
  "version_job_session_id": "ea8e30b7-dd13-4c51-ab38-05b2fa18bbb5"
}


In [3]:
new_version_id = version_response['new_version_id']
metadata = registry.fetch_dataset_metadata(
    registry_endpoint=registry_endpoint, id=new_version_id, auth=get_auth())

updated_metadata = metadata  # "copy" to edit
updated_metadata['dataset_info']["description"] = "This is a new version of the dataset!"
updated_metadata['dataset_info']['spatial_info']['resolution'] = 0.0013
updated_metadata['associations']["point_of_contact"] = "Peter Baker"
updated_metadata['dataset_info']['temporal_info'] = {
    "duration": {
        "begin_date": "2020-07-24",
        "end_date": "2020-07-25"
    },
    "resolution": "P1Y2M10DT2H30M"
}


update_response = datastore.update_dataset(datastore_endpoint=data_store_endpoint,
                                           updated_metadata=updated_metadata, dataset_id=dataset_id, reason=reason, auth=get_auth())
pprint_json(update_response)

Fetching from registry, id: 10378.1/1764778...
Updating dataset with metadata: {'associations': {'organisation_id': '10378.1/1764284', 'data_custodian_id': '10378.1/1758949', 'point_of_contact': 'Peter Baker'}, 'approvals': {'ethics_registration': {'relevant': False, 'obtained': False}, 'ethics_access': {'relevant': False, 'obtained': False}, 'indigenous_knowledge': {'relevant': False, 'obtained': False}, 'export_controls': {'relevant': False, 'obtained': False}}, 'dataset_info': {'name': 'The Test Dataset: A Mirror to the Soul of the Software', 'description': 'This is a new version of the dataset!', 'access_info': {'reposited': True, 'uri': None, 'description': None}, 'publisher_id': '10378.1/1764284', 'created_date': '2022-10-02', 'published_date': '2023-10-03', 'license': 'https://creativecommons.org/licenses/by/4.0/', 'purpose': "But why, you might ask, was the Test Dataset so important? Well, dear reader, it served as a mirror reflecting the very essence of the software, exposing 